In [1]:
"""
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. 

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

"""


'\nIn this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. \n\nFor the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.\n\nOnce the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.\n\n'

In [2]:
!pip install bs4


In [4]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [5]:
#Process and convert html data from wikipedia pg (Feb 2020 vers)
URL = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802'
page = requests.get(URL).text
soup = BeautifulSoup(page,'html.parser')
wiki_table=soup.find('table')
#develop dataframe
df = pd.read_html(str(wiki_table))[0]
df.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
# Remove "Borough" with 'Not assigned' values
df2=df[df['Borough'].str.contains("Not assigned") == False].reset_index()
df2.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
#Part 1- Webscraping Toronto Neighbourhoods Table
df2

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,9,M1B,Scarborough,"Malvern, Rouge"
7,11,M3B,North York,Don Mills
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
df2.shape

(103, 4)

In [8]:
import pandas as pd
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
#Rename 1st column to have same header as in df2- Toronto Neighbourhoods dataframe
df_geo = df_geo.rename(columns = {'PostalCode':'Postal Code'}) 
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df3 = pd.merge(df2, df_geo, on='Postal Code')
df3.head()

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
#Part 2- Toronto neighborhoods dataframe with geocoder latlong data
df3

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,11,M3B,North York,Don Mills,43.745906,-79.352188
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [12]:
# Learn the different value of Borough 
df3['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 4
Toronto/York         1
Mississauga          1
Name: Borough, dtype: int64

In [13]:
# Create a new column as Label and get the data from 'Borough' as integer
df3['Label']=df3['Borough'].replace(to_replace=['North York','Downtown Toronto','Scarborough','Etobicoke','Central Toronto','West Toronto','East York','East Toronto','York','Toronto/York', 'Mississauga'],value=[1,2,3,4,5,6,7,8,9,10,11],inplace=False)
df3.head()


,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Label
0,2,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,3,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2


In [14]:
df3


,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Label
0,2,M3A,North York,Parkwoods,43.753259,-79.329656,1
1,3,M4A,North York,Victoria Village,43.725882,-79.315572,1
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2
5,8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,4
6,9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3
7,11,M3B,North York,Don Mills,43.745906,-79.352188,1
8,12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,7
9,13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2


In [15]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'Geographical coordinates of Toronto: {latitude}, {longitude}.')

Geographical coordinates of Toronto: 43.6534817, -79.3839347.


In [16]:
for data in df3.index:
    borough = df3['Borough'][data]
    neighborhood = df3['Neighbourhood'][data]
    

In [17]:
#Setting the cluster number as number of labels
kclusters=len(df3.Label.unique())

# create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add markers to the map
markers_colors = []
for lat, lon, cluster, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Label'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}'.format(borough + ':', neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

In [18]:
map_toronto